# Step 02: Control Totals

This notebook executes control totals SQL scripts and compares Working Table (3a) vs Contract Import File (3b) results.

**Tasks:**
- Load configuration metadata (date_value, cycle_type)
- Execute 3a_Control_Totals_Working_Table.sql
- Execute 3b_Control_Totals_Contract_Import_File_Tables.sql
- Compare 3a vs 3b results and display differences

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.database import execute_query
from helpers.sqlserver import execute_query_from_file, sql_file_exists
from helpers.constants import WORKSPACE_PATH
from helpers.excel_export import save_control_totals_comparison_to_excel

# Flag to track execution state
execution_failed = False
error_message = None

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_02_Control_Totals.ipynb')

# Display context
ux.header("Control Totals Execution")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"\u2713 Step tracking initialized for '{context.step_name}'")

## 2) Load Configuration

In [ ]:
# Load configuration to get date_value and cycle_type
ux.subheader("Load Configuration")

# Query for active configuration
query = """
    SELECT c.configuration_data
    FROM irp_configuration c
    INNER JOIN irp_cycle cy ON c.cycle_id = cy.id
    WHERE cy.cycle_name = %s
      AND c.status IN ('VALID', 'ACTIVE')
    ORDER BY c.created_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    execution_failed = True
    error_message = "No valid configuration found for this cycle"
    ux.error(f"\u2717 {error_message}")
else:
    config_data = result.iloc[0]['configuration_data']
    metadata = config_data.get('Metadata', {})
    
    date_value = metadata.get('Current Date Value', '')
    cycle_type = metadata.get('Cycle Type', '')
    
    if not date_value or not cycle_type:
        execution_failed = True
        error_message = f"Missing required metadata: date_value={date_value}, cycle_type={cycle_type}"
        ux.error(f"\u2717 {error_message}")
    else:
        ux.success(f"\u2713 Configuration loaded")
        ux.info(f"  Date Value: {date_value}")
        ux.info(f"  Cycle Type: {cycle_type}")
        step.log(f"Configuration loaded: date_value={date_value}, cycle_type={cycle_type}")

## 3) Execute Working Table Control Totals (3a)

In [ ]:
# Execute 3a_Control_Totals_Working_Table.sql
working_table_results = []

if execution_failed:
    ux.warning("\u23ed Skipping 3a execution due to configuration error")
else:
    ux.subheader("Execute Working Table Control Totals")
    
    sql_file_3a = WORKSPACE_PATH / 'sql' / 'control_totals' / '3a_Control_Totals_Working_Table.sql'
    
    if not sql_file_exists(sql_file_3a):
        ux.warning(f"\u26a0 SQL file not found: {sql_file_3a}")
        ux.info("Skipping 3a execution")
    else:
        ux.info(f"Executing: {sql_file_3a.name}")
        ux.info(f"Parameters: DATE_VALUE={date_value}")
        ux.info("")
        
        try:
            # Execute SQL script - returns list of DataFrames (one per SELECT)
            working_table_results = execute_query_from_file(
                sql_file_3a,
                params={'DATE_VALUE': date_value},
                connection='ASSURANT',
                database='DW_EXP_MGMT_USER'
            )
            
            ux.success(f"\u2713 Executed 3a script: {len(working_table_results)} result set(s)")
            step.log(f"Executed 3a script: {len(working_table_results)} result sets")
            
        except Exception as e:
            ux.error(f"\u2717 Error executing 3a script: {str(e)}")
            step.log(f"Error executing 3a script: {str(e)}")

## 4) Execute Import File Control Totals (3b)

In [ ]:
# Execute 3b_Control_Totals_Contract_Import_File_Tables.sql
import_file_results = []

if execution_failed:
    ux.warning("\u23ed Skipping 3b execution due to configuration error")
else:
    ux.subheader("Execute Import File Control Totals")
    
    sql_file_3b = WORKSPACE_PATH / 'sql' / 'control_totals' / '3b_Control_Totals_Contract_Import_File_Tables.sql'
    
    if not sql_file_exists(sql_file_3b):
        ux.warning(f"\u26a0 SQL file not found: {sql_file_3b}")
        ux.info("Skipping 3b execution")
    else:
        ux.info(f"Executing: {sql_file_3b.name}")
        ux.info(f"Parameters: DATE_VALUE={date_value}, CYCLE_TYPE={cycle_type}")
        ux.info("")
        
        try:
            # Execute SQL script - returns list of DataFrames (one per SELECT)
            import_file_results = execute_query_from_file(
                sql_file_3b,
                params={'DATE_VALUE': date_value, 'CYCLE_TYPE': cycle_type},
                connection='ASSURANT',
                database='DW_EXP_MGMT_USER'
            )
            
            ux.success(f"\u2713 Executed 3b script: {len(import_file_results)} result set(s)")
            step.log(f"Executed 3b script: {len(import_file_results)} result sets")
            
        except Exception as e:
            ux.error(f"\u2717 Error executing 3b script: {str(e)}")
            step.log(f"Error executing 3b script: {str(e)}")

## 5) Compare Working Table (3a) vs Import File (3b) Control Totals

Compares control totals between 3a (Working Table) and 3b (Contract Import File):

**Non-Flood perils** (7 attributes): PolicyCount, PolicyPremium, PolicyLimit, LocationCountDistinct, TotalReplacementValue, LocationLimit, LocationDeductible

**Flood perils** (10 attributes): Adds AttachmentPoint, PolicyDeductible, PolicySublimit

**A difference of 0 means the values match.** Detailed results are exported to Excel.

In [ ]:
# Compare 3a vs 3b control totals
from helpers.control_totals import compare_3a_vs_3b_pivot

comparison_results = None
all_matched = False
non_flood_summary = {}
flood_summary = {}

if execution_failed:
    ux.warning("⏭ Skipping comparison due to configuration error")
elif not working_table_results or not import_file_results:
    ux.warning("⚠ Cannot compare: Missing 3a or 3b results")
else:
    ux.subheader("3a vs 3b Comparison Results")
    
    try:
        # Run comparison (pivot format - one row per ExposureGroup)
        comparison_results, all_matched = compare_3a_vs_3b_pivot(
            working_table_results,
            import_file_results
        )
        
        if comparison_results is not None and not comparison_results.empty:
            # Split into Flood and Non-Flood for separate summaries
            is_flood = comparison_results['ExposureGroup'].str.startswith('USFL_')
            flood_results = comparison_results[is_flood]
            non_flood_results = comparison_results[~is_flood]
            
            # Non-Flood summary
            if not non_flood_results.empty:
                non_flood_total = len(non_flood_results)
                non_flood_matched = (non_flood_results['Status'] == 'MATCH').sum()
                non_flood_mismatched = non_flood_total - non_flood_matched
                non_flood_summary = {
                    'total': non_flood_total,
                    'matched': int(non_flood_matched),
                    'mismatched': int(non_flood_mismatched)
                }
                
                ux.info("Non-Flood Perils:")
                if non_flood_mismatched == 0:
                    ux.success(f"  ✓ All {non_flood_total} exposure groups match")
                else:
                    ux.warning(f"  ⚠ {non_flood_mismatched} of {non_flood_total} exposure groups have mismatches")
            
            # Flood summary
            if not flood_results.empty:
                flood_total = len(flood_results)
                flood_matched = (flood_results['Status'] == 'MATCH').sum()
                flood_mismatched = flood_total - flood_matched
                flood_summary = {
                    'total': flood_total,
                    'matched': int(flood_matched),
                    'mismatched': int(flood_mismatched)
                }
                
                ux.info("Flood Perils (USFL_*):")
                if flood_mismatched == 0:
                    ux.success(f"  ✓ All {flood_total} exposure groups match")
                else:
                    ux.warning(f"  ⚠ {flood_mismatched} of {flood_total} exposure groups have mismatches")
            
            # Log to step
            total_groups = len(comparison_results)
            matched_groups = (comparison_results['Status'] == 'MATCH').sum()
            step.log(f"Comparison complete: {matched_groups}/{total_groups} groups matched")
        else:
            ux.warning("No comparison results generated")
            
    except Exception as e:
        ux.error(f"✗ Error comparing control totals: {str(e)}")
        step.log(f"Error comparing control totals: {str(e)}")

### Export Comparison Results to Excel

In [ ]:
# Export comparison results to Excel
excel_path = None

if comparison_results is not None and not comparison_results.empty:
    # Get the notebook directory for output
    notebook_dir = context.notebook_path.parent
    
    excel_path = save_control_totals_comparison_to_excel(
        comparison_results=comparison_results,
        date_value=date_value,
        output_dir=notebook_dir
    )
    
    if excel_path:
        ux.success(f"✓ Comparison results exported to: {excel_path.name}")
        step.log(f"Comparison results exported to: {excel_path}")
else:
    ux.info("No comparison results to export")

## 6) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

if execution_failed:
    # Handle configuration/execution failure
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("CONTROL TOTALS EXECUTION FAILED")
    ux.error("="*60)
    ux.error(f"\nError: {error_message}")
    ux.info("\nPlease fix the error and retry.")

else:
    # Build comparison summary for output
    comparison_summary = {}
    if comparison_results is not None and not comparison_results.empty:
        total_groups = len(comparison_results)
        matched_groups = int((comparison_results['Status'] == 'MATCH').sum())
        comparison_summary = {
            'total_exposure_groups': total_groups,
            'matched_groups': matched_groups,
            'mismatched_groups': total_groups - matched_groups,
            'all_matched': bool(all_matched),
            'non_flood': non_flood_summary,
            'flood': flood_summary
        }
    
    # Complete the step successfully
    output_data = {
        'date_value': date_value,
        'cycle_type': cycle_type,
        'working_table_result_count': len(working_table_results),
        'import_file_result_count': len(import_file_results),
        'comparison': comparison_summary
    }
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("CONTROL TOTALS EXECUTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nWorking Table (3a): {len(working_table_results)} result set(s)")
    ux.info(f"Import File (3b): {len(import_file_results)} result set(s)")
    
    if comparison_summary:
        ux.info(f"\n3a vs 3b Comparison:")
        
        # Non-Flood summary
        if non_flood_summary:
            if non_flood_summary['mismatched'] == 0:
                ux.success(f"  Non-Flood: ✓ All {non_flood_summary['total']} groups match")
            else:
                ux.warning(f"  Non-Flood: ⚠ {non_flood_summary['mismatched']}/{non_flood_summary['total']} groups have mismatches")
        
        # Flood summary
        if flood_summary:
            if flood_summary['mismatched'] == 0:
                ux.success(f"  Flood: ✓ All {flood_summary['total']} groups match")
            else:
                ux.warning(f"  Flood: ⚠ {flood_summary['mismatched']}/{flood_summary['total']} groups have mismatches")
    
    if excel_path:
        ux.info(f"\nDetailed results exported to: {excel_path.name}")
    
    ux.info("\nNext: Proceed to Stage 03 Data Import")

    # Send Teams notification for Stage 02 completion
    try:
        from helpers.teams_notification import TeamsNotificationClient, build_notification_actions
        from helpers.database import get_current_schema

        teams = TeamsNotificationClient()
        schema = get_current_schema()
        actions = build_notification_actions(
            notebook_path=str(context.notebook_path),
            cycle_name=context.cycle_name,
            schema=schema
        )
        
        # Build comparison status message
        comparison_msg = ""
        if comparison_summary:
            if non_flood_summary:
                if non_flood_summary['mismatched'] == 0:
                    comparison_msg += f"- Non-Flood: ✓ All {non_flood_summary['total']} groups match\n"
                else:
                    comparison_msg += f"- Non-Flood: ⚠ {non_flood_summary['mismatched']}/{non_flood_summary['total']} groups have mismatches\n"
            if flood_summary:
                if flood_summary['mismatched'] == 0:
                    comparison_msg += f"- Flood: ✓ All {flood_summary['total']} groups match\n"
                else:
                    comparison_msg += f"- Flood: ⚠ {flood_summary['mismatched']}/{flood_summary['total']} groups have mismatches\n"

        teams.send_success(
            title=f"[{context.cycle_name}] Stage 02 Data Extraction Complete",
            message=f"**Cycle:** {context.cycle_name}\n"
                    f"**Stage:** {context.stage_name}\n"
                    f"**Step:** {context.step_name}\n\n"
                    f"**Results:**\n"
                    f"- Working Table (3a): {len(working_table_results)} result set(s)\n"
                    f"- Import File (3b): {len(import_file_results)} result set(s)\n\n"
                    f"**3a vs 3b Comparison:**\n"
                    f"{comparison_msg}\n"
                    f"Ready to proceed to Stage 03 Data Import.",
            actions=actions
        )
        ux.info("\nTeams notification sent.")
    except Exception as e:
        ux.warning(f"\nCould not send Teams notification: {str(e)}")